In [1]:
%display latex

%runfile young_idempotent.py
%runfile diagonal_polynomial_ring.py
%runfile subspace.py
%runfile add_degree.py
%runfile diagram.py

SymmetricFunctions(QQ).inject_shorthands(verbose=False)

In [1]:
%display latex
%runfile character.py

# Code

Calcul du Vandermonde associé au diagramme $\gamma$, où $\gamma$ est un partage (Partition) ou un diagramme (Diagram).

In [2]:
def vandermonde(gamma, r=0):
    n = gamma.size()
    if r == 0:
        r = 1
    P = DiagonalPolynomialRing(QQ, n, r, inert=1) 
    X = P.variables()
    Theta = P.inert_variables()
    return matrix([[x**i[1]*theta**i[0] for i in gamma.cells()] 
                   for x,theta in zip(X[0],Theta[0])]).determinant()

Fonctions qui nous renvoient des listes d'opérateurs par degrés pour toutes les variables (non inertes) de l'anneau de polynômes concerné.

In [3]:
def partial_derivatives(P):
    """
    Return the partial derivative functions in all the variables of `P`.
    
    INPUT:
    - `P` -- a diagonal polynomial ring
    
    """
    n = P.ncols()
    r = P.nrows()
    D = P.grading_set()
    X = P.derivative_variables()
    op = {}
    for i in range(r):
        op[D((-1 if j==i else 0 for j in range(r)))] = [attrcall("derivative", X[i,k]) for k in range(n)]
    return op

def polarization_operators(r, max_deg=1, side=None, row_symmetry=None):
    D = cartesian_product([ZZ for i in range(r)])
    return {D([-d if i==i1 else 1 if i==i2 else 0 for i in range(r)]):
            [attrcall("polarization", i1=i1, i2=i2, d=d, row_symmetry=row_symmetry)]
            for d in range(1, max_deg+1)
            for i1 in range(0, r)
            for i2 in range(0, r)
            if (i1<i2 if side == 'down' else i1!=i2)
           }

##############################################################

def steenrod_operators(r, degree=1, row_symmetry=None):
    D = cartesian_product([ZZ for i in range(r)])
    op = {}
    for i in range(r):
        op[D((-degree if j==i else 0 for j in range(r)))] = [
            attrcall("steenrod_op", i=i, k=degree+1, row_symmetry=row_symmetry)]
    return op

def diagonal_steenrod_operators(list_deg):
    r = len(list_deg[0])
    D = cartesian_product([ZZ for i in range(r)])
    op = {}
    for dx, dy in list_deg:
        op[D((-dx if j==0 else -dy if j==1 else 0 for j in range(r)))] = [
            attrcall("diagonal_steenrod", i1=0, i2=1, d1=dx, d2=dy)]
    return op

def higher_polarization_operators(r, max_deg=1, side=None, row_symmetry=None):
    D = cartesian_product([ZZ for i in range(r)])
    return {D([-d1 if i==i1 else d2 if i==i2 else 0 for i in range(r)]):
            [attrcall("higher_polarization", i1=i1, i2=i2, d1=d1, d2=d2, row_symmetry=row_symmetry)]
            for d1 in range(1, max_deg+1)
            for d2 in range(1, 3)
            for i1 in range(0, r)
            for i2 in range(0, r)
            if (i1<i2 if side == 'down' else i1!=i2)
           }

def multipolarization(list_deg, i2):
    r = len(list_deg[0])
    D = cartesian_product([ZZ for i in range(r)])
    return {D(-d[i]+1 if i==i2 else -d[i] for i in range(len(d))) : 
            [attrcall("multi_polarization", D=d, i2=i2)] for d in list_deg}

def symmetric_derivatives(list_deg, row_symmetry=None):
    r = len(list_deg[0])
    D = cartesian_product([ZZ for i in range(r)])
    return {D(-i for i in d) : [attrcall("symmetric_derivative", d=d, row_symmetry=row_symmetry)] 
            for d in list_deg}

In [4]:
def merge(dict1, dict2):
    result = dict1
    for key, value in dict2.iteritems():
        if key in result:
            result[key] += value
        else:
            result[key] = value
    return result

def antisymmetries(mu):
    mu = Partition(mu)
    return antisymmetries_of_tableau(mu.initial_tableau())

Projection sur les composantes isotypiques; prend en paramètre un sous-espace (Subspace) et un entier ou un partage. 

In [5]:
def IsotypicComponent(S, arg, use_antisymmetry=False):
    if isinstance(arg, Partition):
        list_partitions = [arg]
    elif isinstance(arg, Integer):
        list_partitions = Partitions(arg)
    else : 
        print("Error: arg should be a partition or an integer.")
    
    basis = S.basis()
    result = {}
    P1 = basis.values().pop()[0].parent()
    for nu in list_partitions:
        result_nu = {}
        if use_antisymmetry == True:
            antisymmetries = antisymmetries_of_tableau(nu.initial_tableau())
            P2 = DiagonalAntisymmetricPolynomialRing(P1._R, P1.ncols(), P1.nrows(), 
                                                 P1.ninert(), antisymmetries)
        for deg, value in basis.iteritems():
            if use_antisymmetry:
                gen = []
                for p in value:
                    temp = apply_young_idempotent(P2(p), nu)
                    if temp != 0: 
                        gen += [temp]
            else:
                gen = []
                for p in value:
                    temp = apply_young_idempotent(p, nu)
                    if temp != 0:
                        gen += [temp]
            if gen != []:
                result_nu[(deg, tuple(nu))] = Subspace(gen, {}).basis()[0]
        if result_nu != {}:
            result[nu] = Subspace(result_nu, operators={})
                
    if len(result.keys()) == 1:
        key = result.keys()[0]
        return result[key]
    else:
        return result

In [6]:
def add_degrees_isotypic(gen_deg, op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    return D(gen_deg[0])+D(op_deg), gen_deg[1]

def add_degrees_symmetric(gen_deg,op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    d = D(gen_deg[0])+D(op_deg)
    return D(sorted(d, reverse=True)), gen_deg[1]

def add_degrees_test(gen_deg, op_deg):
    return gen_deg+op_deg

Création de l'espace polarisé par les opérateurs donnés en paramètres.

In [7]:
def PolarizedSpace(S, operators, add_degrees=add_degrees_isotypic):
    if isinstance(S, dict):
        return {key : PolarizedSpace(value, operators, add_degrees=add_degrees)
                for key, value in S.iteritems()}
    else:
        basis = S.basis()
        basis_element = basis.values().pop()[0]
        P1 = basis_element.parent()
        r = len(op_pol.keys().pop())
        row_symmetry = op_pol.values().pop()[0].kwds['row_symmetry']
        if row_symmetry == "permutation":
            add_degrees = add_degrees_symmetric
        D = cartesian_product([ZZ for i in range(r)])
        generators = {}

        if isinstance(P1, DiagonalAntisymmetricPolynomialRing):
            P2 = DiagonalAntisymmetricPolynomialRing(P1._R, P1.ncols(), r , P1.ninert(), P1.antisymmetries())
            for key, value in basis.iteritems():
                d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
                generators[d] = tuple(reduce_antisymmetric_normal(P2(b), 
                                                      b.parent().ncols(), 
                                                      b.parent().nrows()+b.parent().ninert(), 
                                                      b.antisymmetries()) for b in value)
        else :
            P2 = DiagonalPolynomialRing(P1._R, P1.ncols(), r , P1.ninert())
            for key, value in basis.iteritems():
                d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
                generators[d] = tuple(P2(b) for b in value)
        return Subspace(generators, operators, add_degrees=add_degrees)

Calcul de l'image de S (Subspace) par les opérateurs donnés.

In [8]:
def Range(S, operators, add_degrees=add_degrees_isotypic):
    if isinstance(S, dict):
        return {key : Range(value, operators, add_degrees=add_degrees)
                for key, value in S.iteritems()}

    result = {}
    basis = S.basis()
    for key, b in basis.iteritems():
        result = merge(result, {add_degrees(key, deg): 
                                     [op(p) for p in b for op in op_list if op(p)!=0] 
                                     for deg, op_list in operators.iteritems()})    
    if result != {} :
        return Subspace(result, {}, add_degrees) #{} <-> operators
    else :
        return None

Calcul du caractère à partir d'un sous-espace (S) d'éléments projetés sur les composantes isotypiques.

In [9]:
def character(S, left_basis=s, right_basis=s, row_symmetry=None):
    if isinstance(S, dict):
        return sum(character(V,
                             left_basis=left_basis, right_basis=right_basis, row_symmetry=row_symmetry) 
                   for V in S.values())
    else:
        basis = S.basis()
        basis_element = basis.values().pop()[0]
        P = basis_element.parent()
        n = P.ncols()
        r = P.nrows()
        
        charac = 0
        if row_symmetry != "permutation":
            q = PolynomialRing(QQ,'q',r).gens()

        for nu in Partitions(n):
            basis_nu = {}
            charac_nu = 0
            # Get the nu_isotypic part of the basis
            for key, value in basis.iteritems():
                if Partition(key[1]) == nu:
                    basis_nu[key[0]] = value

            # Use monomials to compute the character
            if row_symmetry == "permutation":
                for deg, b in basis_nu.iteritems():
                    charac_nu += sum(m(Partition(deg)) for p in b)
                if charac_nu != 0 :
                    if left_basis == s :
                        charac_nu = s(charac_nu).restrict_partition_lengths(r,exact=False)
                    elif left_basis != m :
                        charac_nu = left_basis(charac_nu)

            # Or use directly the degrees
            else:
                for deg, b in basis_nu.iteritems():
                    charac_nu += sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
                if charac_nu != 0 :
                    if left_basis == s :
                        charac_nu = s.from_polynomial(charac_nu).restrict_partition_lengths(r,exact=False)           
                    else:
                        charac_nu = left_basis.from_polynomial(charac_nu)

            # Make the tensor product with s[nu]
            if charac_nu != 0:
                charac += tensor([charac_nu, right_basis(s(nu))])
        return charac

In [10]:
def character_quotient(M, N, n, r, left_basis=s, right_basis=s): # a corriger
    b_tot = M.basis()
    b_ideal = N.basis()
    charac = 0
    q = PolynomialRing(QQ,'q',r).gens()
    
    for nu in Partitions(n):
        basis_nu_tot = {}
        basis_nu_ideal = {}
        charac_nu = 0
        # Get the nu_isotypic part of the bases
        for key, value in b_tot.iteritems():
            if Partition(key[1]) == nu:
                basis_nu_tot[key[0]] = value
        for key, value in b_ideal.iteritems():
            if Partition(key[1]) == nu:
                basis_nu_ideal[key[0]] = value
                
        # Use the degrees to compute the character
        for deg, b in basis_nu_tot.iteritems():
            charac_nu += sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
        for deg, b in basis_nu_ideal.iteritems():
            charac_nu -= sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
        if charac_nu != 0 :
            if left_basis == s :
                charac_nu = s.from_polynomial(charac_nu).restrict_partition_lengths(r,exact=False)           
            else:
                charac_nu = left_basis.from_polynomial(charac_nu)      
            # Make the tensor product with s[nu]
            charac += tensor([charac_nu, right_basis(s(nu))])
            
    return charac

Factorisation du produit tensoriel par représentation de $S_n$. Et calcul de la dimension.

In [11]:
def factorize(f, n):
    SymmetricFunctions(QQ).s()
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu])) for mu in result.keys()]
    for a, b in result2:
        if b!=0:
            print a
            show(b)
        
def dimension(f, n):
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu]).expand(1, alphabet=['q'])) for mu in result.keys()]
    q = result2[0][1].parent().gens()[0]
    return [(tuple(a), b.subs({q:1})) for a,b in result2]

def dimension2(f, n):
    q,t = QQ['q,t'].gens()
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu]).expand(2, alphabet=[q,t])) for mu in result.keys()]
    return [(tuple(a), b.subs({q:1,t:1})) for a,b in result2]

In [12]:
def compute_character(mu, use_antisymmetry=True, row_symmetry="permutation"):
    n = Integer(mu.size())
    # Determinant computation
    v = vandermonde(mu)
    # Span by derivatives
    generator = {v.multidegree() : [v]}
    list_op = partial_derivatives(v.parent())
    V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
    # Projection on isotypic components
    V_iso = IsotypicComponent(V, n, use_antisymmetry=use_antisymmetry)
    # Polarization
    r = n-1
    deg = v.degree()
    if deg == 0:
        deg = 1
    op_pol = polarization_operators(r, deg, row_symmetry=row_symmetry)
    V_pol = PolarizedSpace(V_iso, op_pol)
    
    # character
    return character(V_pol, row_symmetry=row_symmetry)

In [13]:
#import pdb

In [14]:
#pdb.pm()

# Exemples de calculs

### Cas du partage 21 - sans antisymétries

Calcul du déterminant de Vandermonde $\mathcal{V}_\lambda$ associé au partage $\lambda = 21$.

In [2]:
n = 3
v = vandermonde(Partition([2,1]))
v

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

Espace généré par $\mathcal{V}_\lambda$ et ses dérivées partielles.

In [3]:
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V.basis()

{(0,): (theta01 - theta02, theta00 - theta02),
 (1,): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

Projection sur les composantes isotypiques.

In [4]:
V_iso = IsotypicComponent(V, n)
for value in V_iso.values():
    show(value.basis())

{((1,),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0,), (2, 1)): (-theta00 + theta02,)}

On ajoute un jeu de variables supplémentaire et on polsarise.

In [200]:
r = 2 
deg = 2
op_pol = polarization_operators(r, deg)
V_pol = PolarizedSpace(V_iso, op_pol)
for value in V_pol.values():
    show(value.basis())

{((0, 1),
  (1,
   1,
   1)): (-x11*theta00 + x12*theta00 + x10*theta01 - x12*theta01 - x10*theta02 + x11*theta02,),
 ((1, 0),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0, 0), (2, 1)): (theta00 - theta02,)}

Calcul du bicaractère.

In [201]:
character(V_pol)

s[] # s[2, 1] + s[1] # s[1, 1, 1]

### Cas du partage 21 - avec antisymétries

In [202]:
v = vandermonde(Partition([2,1]))
v

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

Espace généré par $\mathcal{V}_\lambda$ et ses dérivées partielles.

In [203]:
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V.basis()

{(0,): (theta01 - theta02, theta00 - theta02),
 (1,): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

Projection sur les composantes isotypiques.

In [204]:
V_iso = IsotypicComponent(V, 3, use_antisymmetry=True)
for value in V_iso.values():
    show(value.basis())

{((1,), (1, 1, 1)): (x00*theta01,)}

{((0,), (2, 1)): (theta00,)}

On ajoute un jeu de variables supplémentaire et on polsarise.

In [205]:
r = 2 
deg = 2
op_pol = polarization_operators(r, deg)
V_pol = PolarizedSpace(V_iso, op_pol)
for value in V_pol.values():
    show(value.basis())

{((0, 1), (1, 1, 1)): (x10*theta01,), ((1, 0), (1, 1, 1)): (x00*theta01,)}

{((0, 0), (2, 1)): (theta00,)}

Calcul du bicaractère

In [206]:
character(V_pol)

s[] # s[2, 1] + s[1] # s[1, 1, 1]

### Tout en un

In [207]:
mu = Partition([4,1])
n = mu.size()
v = vandermonde(mu)
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V_iso = IsotypicComponent(V, n, use_antisymmetry=True)

r = n-1
deg = degree_vandermonde(mu)
if deg == 0:
    deg = 1
op_pol = polarization_operators(r, deg, row_symmetry="permutation")
V_pol = PolarizedSpace(V_iso, op_pol)

character(V_pol, row_symmetry="permutation")

s[] # s[4, 1] + s[1] # s[3, 1, 1] + s[1] # s[3, 2] + s[1, 1] # s[2, 1, 1, 1] + s[1, 1] # s[2, 2, 1] + s[1, 1, 1] # s[1, 1, 1, 1, 1] + s[2] # s[2, 2, 1] + s[2] # s[3, 1, 1] + s[2] # s[3, 2] + s[2, 1] # s[2, 1, 1, 1] + s[2, 1] # s[2, 2, 1] + s[3] # s[2, 1, 1, 1] + s[3] # s[2, 2, 1] + s[3] # s[3, 1, 1] + s[3, 1] # s[1, 1, 1, 1, 1] + s[3, 1] # s[2, 1, 1, 1] + s[4] # s[2, 1, 1, 1] + s[4] # s[2, 2, 1] + s[4, 1] # s[1, 1, 1, 1, 1] + s[5] # s[2, 1, 1, 1] + s[6] # s[1, 1, 1, 1, 1]

In [208]:
def compute_caracter(mu, use_antisymmetry=True, row_symmetry="permutation"):
    n = Integer(mu.size())
    # Determinant computation
    v = vandermonde(mu)
    # Span by derivatives
    generator = {v.multidegree() : [v]}
    list_op = partial_derivatives(v.parent())
    V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
    # Projection on isotypic components
    V_iso = IsotypicComponent(V, n, use_antisymmetry=use_antisymmetry)
    # Polarization
    r = n-1
    deg = degree_vandermonde(mu)
    if deg == 0:
        deg = 1
    op_pol = polarization_operators(r, deg, row_symmetry=row_symmetry)
    V_pol = PolarizedSpace(V_iso, op_pol)
    
    # character
    return character(V_pol, row_symmetry=row_symmetry)

In [209]:
compute_character(Partition([4,1]))

s[] # s[4, 1] + s[1] # s[3, 1, 1] + s[1] # s[3, 2] + s[1, 1] # s[2, 1, 1, 1] + s[1, 1] # s[2, 2, 1] + s[1, 1, 1] # s[1, 1, 1, 1, 1] + s[2] # s[2, 2, 1] + s[2] # s[3, 1, 1] + s[2] # s[3, 2] + s[2, 1] # s[2, 1, 1, 1] + s[2, 1] # s[2, 2, 1] + s[3] # s[2, 1, 1, 1] + s[3] # s[2, 2, 1] + s[3] # s[3, 1, 1] + s[3, 1] # s[1, 1, 1, 1, 1] + s[3, 1] # s[2, 1, 1, 1] + s[4] # s[2, 1, 1, 1] + s[4] # s[2, 2, 1] + s[4, 1] # s[1, 1, 1, 1, 1] + s[5] # s[2, 1, 1, 1] + s[6] # s[1, 1, 1, 1, 1]

### Cas du diagramme 013

In [210]:
v = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
v.factor()

(x01 - x02) * (-x00 + x01) * (x00 - x02) * (x00 + x01 + x02)

In [211]:
v.parent()

Diagonal Polynomial Ring with 1 rows of 3 variables and inert variables over Rational Field

On crée le sous-espace engendré par $\mathcal{V}_\gamma$ et ses dérivées partielles. On peut ajouter les opérateurs de Steenrod sans changement du résultat.

In [212]:
generator = {v.multidegree() : [v]}
list_op = merge(merge(partial_derivatives(v.parent()), steenrod_operators(1, 1)), steenrod_operators(1, 2))

In [213]:
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (x00^2 - x02^2, x01^2 - x02^2, -x00*x01 + x00*x02, -x00*x01 + x01*x02),
 (3,): (1/3*x00^3 - 1/3*x01^3 - x00*x02^2 + x01*x02^2,
  -1/3*x00^3 + x00*x01^2 - x01^2*x02 + 1/3*x02^3,
  -x00^2*x01 + 1/3*x01^3 + x00^2*x02 - 1/3*x02^3),
 (4,): (-x00^3*x01 + x00*x01^3 + x00^3*x02 - x01^3*x02 - x00*x02^3 + x01*x02^3,)}

In [214]:
character(IsotypicComponent(W1, 3))

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Appliquons $\sum_i \partial_{x_i}$ et $\sum_i x_i \partial_{x_i}^2$ à $\mathcal{V}_{\gamma}$. 

In [215]:
x00, x01, x02 = P1.variables()[0]
v.symmetric_derivative((1,))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [216]:
v.steenrod_op(0, 2)

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

Polarisation

In [217]:
op_pol = polarization_operators(2, max_deg=v.degree())
W2 = PolarizedSpace(IsotypicComponent(W1, 3), op_pol)
character(W2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

In [218]:
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(P2._r)) 
                for k1 in range(1, 3+1) for i1 in range(0, P2._r)]
#list_degrees += [(1,1),(1,2),(1,3),(2,1),(2,2),(2,3),(3,1),(3,2),(3,3)]
sym_diff = symmetric_derivatives(2, list_degrees)
#sym_diff = merge(merge(symmetric_derivatives(P2, list_degrees), 
#                       steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
character(Range(W2, sym_diff))

s[] # s[3] + s[1] # s[2, 1] + s[1, 1] # s[1, 1, 1] + s[2] # s[2, 1] + s[3] # s[1, 1, 1]

In [219]:
charac = character(W2) - character(Range(W2, sym_diff))
charac

s[1] # s[3] + s[1, 1] # s[2, 1] + s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Ou dans la base des monomiales

In [220]:
character(W2, left_basis=m) - character(Range(W2, sym_diff), left_basis=m)

m[1] # s[3] + 2*m[1, 1] # s[2, 1] + m[2] # s[2, 1] + m[2, 1] # s[1, 1, 1] + m[2, 1] # s[2, 1] + m[2, 2] # s[1, 1, 1] + m[3] # s[2, 1] + m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]

On factorise le caractère par représentation de $S_3$.

In [221]:
factorise(charac, 3)

[1, 1, 1]


s[2, 1] + s[4]

[3]


s[1]

[2, 1]


s[1, 1] + s[2] + s[3]

### Cas du partage 311

In [253]:
v = vandermonde(Partition([3,1,1]))
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(2, max_deg=deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, 5), op_pol)
character(W2)

s[] # s[3, 1, 1] + s[1] # s[2, 1, 1, 1] + s[1] # s[2, 2, 1] + s[1, 1] # s[1, 1, 1, 1, 1] + s[2] # s[2, 1, 1, 1] + s[3] # s[1, 1, 1, 1, 1]

### Exemples avec et sans les antisymétries

Comparons les bases sur un petit exemple sans puis avec l'utilisation des antisymétries.

In [254]:
for nu in Partitions(3):
    print nu
    show(vandermonde(nu))
    show(antisymmetries(nu))

[3]


-x00^2*x01 + x00*x01^2 + x00^2*x02 - x01^2*x02 - x00*x02^2 + x01*x02^2

((0,), (1,), (2,))

[2, 1]


-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

((0, 2), (1,))

[1, 1, 1]


-theta00^2*theta01 + theta00*theta01^2 + theta00^2*theta02 - theta01^2*theta02 - theta00*theta02^2 + theta01*theta02^2

((0, 1, 2),)

In [255]:
mu = Partition([2,1])
n = 3
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(2, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1] + s[1] # s[1, 1, 1]

{((0, 1),
  (1,
   1,
   1)): (-x11*theta00 + x12*theta00 + x10*theta01 - x12*theta01 - x10*theta02 + x11*theta02,),
 ((1, 0),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0, 0), (2, 1)): (theta00 - theta02,)}

In [256]:
mu = Partition([2,1])
n = 3
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol)
show(character(W2))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1] + s[1] # s[1, 1, 1]

{((0, 1), (1, 1, 1)): (x10*theta01,), ((1, 0), (1, 1, 1)): (x00*theta01,)}

{((0, 0), (2, 1)): (theta00,)}

Comparaison des temps de calcul avec et sans les antisymétries sur un exemple plus gros

In [257]:
import datetime

In [258]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 1, 667741)

In [259]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol)
show(character(W2))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 0, 735903)

### Exemples avec et sans les symétries sur les lignes

Comparons les bases sur un petit exemple sans puis avec l'utilisation des symétries sur les lignes.

In [260]:
mu = Partition([2,1,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]

{((0, 0, 0),
  (2,
   1,
   1)): (theta00^2*theta02 - theta00*theta02^2 - theta00^2*theta03 + theta02^2*theta03 + theta00*theta03^2 - theta02*theta03^2,)}

{((0, 0, 1),
  (1,
   1,
   1,
   1)): (-x22*theta00^2*theta01 + x23*theta00^2*theta01 + x22*theta00*theta01^2 - x23*theta00*theta01^2 + x21*theta00^2*theta02 - x23*theta00^2*theta02 - x20*theta01^2*theta02 + x23*theta01^2*theta02 - x21*theta00*theta02^2 + x23*theta00*theta02^2 + x20*theta01*theta02^2 - x23*theta01*theta02^2 - x21*theta00^2*theta03 + x22*theta00^2*theta03 + x20*theta01^2*theta03 - x22*theta01^2*theta03 - x20*theta02^2*theta03 + x21*theta02^2*theta03 + x21*theta00*theta03^2 - x22*theta00*theta03^2 - x20*theta01*theta03^2 + x22*theta01*theta03^2 + x20*theta02*theta03^2 - x21*theta02*theta03^2,),
 ((0, 1, 0),
  (1,
   1,
   1,
   1)): (-x12*theta00^2*theta01 + x13*theta00^2*theta01 + x12*theta00*theta01^2 - x13*theta00*theta01^2 + x11*theta00^2*theta02 - x13*theta00^2*theta02 - x10*theta01^2*theta02 + x13*theta01^2*theta02 - x11*theta00*theta02^2 + x13*theta00*theta02^2 + x10*theta01*theta02^2 - x13*theta01*theta02^2 - x11*theta00^2*theta03 + x12*theta00^2*theta03 + x10*theta01^2*theta03 - x12*theta01^2*theta03 - x10*theta02^2*theta03 + x11*theta02^2*theta03 + x11*theta00*theta03^2 - x12*theta00*theta03^2 - x10*theta01*theta03^2 + x12*theta01*theta03^2 + x10*theta02*theta03^2 - x11*theta02*theta03^2,),
 ((1, 0, 0),
  (1,
   1,
   1,
   1)): (-x02*theta00^2*theta01 + x03*theta00^2*theta01 + x02*theta00*theta01^2 - x03*theta00*theta01^2 + x01*theta00^2*theta02 - x03*theta00^2*theta02 - x00*theta01^2*theta02 + x03*theta01^2*theta02 - x01*theta00*theta02^2 + x03*theta00*theta02^2 + x00*theta01*theta02^2 - x03*theta01*theta02^2 - x01*theta00^2*theta03 + x02*theta00^2*theta03 + x00*theta01^2*theta03 - x02*theta01^2*theta03 - x00*theta02^2*theta03 + x01*theta02^2*theta03 + x01*theta00*theta03^2 - x02*theta00*theta03^2 - x00*theta01*theta03^2 + x02*theta01*theta03^2 + x00*theta02*theta03^2 - x01*theta02*theta03^2,)}

In [261]:
mu = Partition([2,1,1])
n = 4
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, max_deg=deg_v, row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol, add_degrees=add_degrees_symmetric)
show(character(W2, row_symmetry="permutation"))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]

{((0, 0, 0),
  (2,
   1,
   1)): (theta00^2*theta02 - theta00*theta02^2 - theta00^2*theta03 + theta02^2*theta03 + theta00*theta03^2 - theta02*theta03^2,)}

{((1, 0, 0),
  (1,
   1,
   1,
   1)): (-x02*theta00^2*theta01 + x03*theta00^2*theta01 + x02*theta00*theta01^2 - x03*theta00*theta01^2 + x01*theta00^2*theta02 - x03*theta00^2*theta02 - x00*theta01^2*theta02 + x03*theta01^2*theta02 - x01*theta00*theta02^2 + x03*theta00*theta02^2 + x00*theta01*theta02^2 - x03*theta01*theta02^2 - x01*theta00^2*theta03 + x02*theta00^2*theta03 + x00*theta01^2*theta03 - x02*theta01^2*theta03 - x00*theta02^2*theta03 + x01*theta02^2*theta03 + x01*theta00*theta03^2 - x02*theta00*theta03^2 - x00*theta01*theta03^2 + x02*theta01*theta03^2 + x00*theta02*theta03^2 - x01*theta02*theta03^2,)}

Avec les antisymétries aussi. 

In [262]:
mu = Partition([2,1,1])
n = 4
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, max_deg=deg_v, row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol, add_degrees=add_degrees_symmetric)
show(character(W2, row_symmetry="permutation"))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]

{((0, 0, 0), (2, 1, 1)): (theta00^2*theta02,)}

{((1, 0, 0), (1, 1, 1, 1)): (x00*theta01^2*theta02,)}

Comparaison des temps de calcul avec et sans les symétries sur un exemple plus gros

In [263]:
import datetime

In [264]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 1, 772244)

In [265]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, max_deg=deg_v, row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol, add_degrees=add_degrees_symmetric)
show(character(W2, row_symmetry="permutation"))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 1, 209982)

In [266]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, max_deg=deg_v, row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol, add_degrees=add_degrees_symmetric)
show(character(W2, row_symmetry="permutation"))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 0, 584504)

### Cas $m=4$, $n=6$

In [267]:
v = vandermonde(Diagram([(0,0),(1,0),(2,0),(3,0),(0,1),(2,1)]))
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(3, max_deg=deg_v, row_symmetry="permutation")
#W2 = PolarizedSpace(IsotypicComponent(W1, 6, use_antisymmetry=True), op_pol)
#character(W2, row_symmetry="permutation")

### Autres cas

#### Diagramme $\{(0,0),(0,1),(1,1),(0,2)\}$ et autres

Pas de quotient dans ce cas.

In [56]:
def f(gamma):
    v = vandermonde(gamma)
    n = gamma.size()
    deg_v = v.degree()
    generator = {v.multidegree() : [v]}
    list_op = partial_derivatives(v.parent())
    W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

    r = 1
    op_pol = polarization_operators(r, deg_v, row_symmetry="permutation")
    show(IsotypicComponent(W1, Partition([1 for i in range(n)])).basis())
    show(IsotypicComponent(W1, gamma).basis())
    W1 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol)
    return character(W1, row_symmetry="permutation")

In [57]:
f(Partition([2]))

{((1,), (1, 1)): (-x00 + x01,)}

{((0,), (2,)): (1,)}

s[] # s[2] + s[1] # s[1, 1]

In [58]:
f(Partition([2,1]))

{((1,),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0,), (2, 1)): (-theta00 + theta02,)}

s[] # s[2, 1] + s[1] # s[1, 1, 1]

In [59]:
f(Partition([2,1,1]))

{((1,),
  (1,
   1,
   1,
   1)): (x02*theta00^2*theta01 - x03*theta00^2*theta01 - x02*theta00*theta01^2 + x03*theta00*theta01^2 - x01*theta00^2*theta02 + x03*theta00^2*theta02 + x00*theta01^2*theta02 - x03*theta01^2*theta02 + x01*theta00*theta02^2 - x03*theta00*theta02^2 - x00*theta01*theta02^2 + x03*theta01*theta02^2 + x01*theta00^2*theta03 - x02*theta00^2*theta03 - x00*theta01^2*theta03 + x02*theta01^2*theta03 + x00*theta02^2*theta03 - x01*theta02^2*theta03 - x01*theta00*theta03^2 + x02*theta00*theta03^2 + x00*theta01*theta03^2 - x02*theta01*theta03^2 - x00*theta02*theta03^2 + x01*theta02*theta03^2,)}

{((0,),
  (2,
   1,
   1)): (-theta00^2*theta02 + theta00*theta02^2 + theta00^2*theta03 - theta02^2*theta03 - theta00*theta03^2 + theta02*theta03^2,)}

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]

In [61]:
f(Partition([2,2,1,1]))

{((2,),
  (1,
   1,
   1,
   1,
   1,
   1)): (x02*x04*theta00^3*theta01^2*theta02*theta03 - x03*x04*theta00^3*theta01^2*theta02*theta03 - x02*x05*theta00^3*theta01^2*theta02*theta03 + x03*x05*theta00^3*theta01^2*theta02*theta03 - x02*x04*theta00^2*theta01^3*theta02*theta03 + x03*x04*theta00^2*theta01^3*theta02*theta03 + x02*x05*theta00^2*theta01^3*theta02*theta03 - x03*x05*theta00^2*theta01^3*theta02*theta03 - x01*x04*theta00^3*theta01*theta02^2*theta03 + x03*x04*theta00^3*theta01*theta02^2*theta03 + x01*x05*theta00^3*theta01*theta02^2*theta03 - x03*x05*theta00^3*theta01*theta02^2*theta03 + x00*x04*theta00*theta01^3*theta02^2*theta03 - x03*x04*theta00*theta01^3*theta02^2*theta03 - x00*x05*theta00*theta01^3*theta02^2*theta03 + x03*x05*theta00*theta01^3*theta02^2*theta03 + x01*x04*theta00^2*theta01*theta02^3*theta03 - x03*x04*theta00^2*theta01*theta02^3*theta03 - x01*x05*theta00^2*theta01*theta02^3*theta03 + x03*x05*theta00^2*theta01*theta02^3*theta03 - x00*x04*theta00*theta01^2*theta02^3*theta03 + x03*x04*theta00*theta01^2*theta02^3*theta03 + x00*x05*theta00*theta01^2*theta02^3*theta03 - x03*x05*theta00*theta01^2*theta02^3*theta03 + x01*x04*theta00^3*theta01*theta02*theta03^2 - x02*x04*theta00^3*theta01*theta02*theta03^2 - x01*x05*theta00^3*theta01*theta02*theta03^2 + x02*x05*theta00^3*theta01*theta02*theta03^2 - x00*x04*theta00*theta01^3*theta02*theta03^2 + x02*x04*theta00*theta01^3*theta02*theta03^2 + x00*x05*theta00*theta01^3*theta02*theta03^2 - x02*x05*theta00*theta01^3*theta02*theta03^2 + x00*x04*theta00*theta01*theta02^3*theta03^2 - x01*x04*theta00*theta01*theta02^3*theta03^2 - x00*x05*theta00*theta01*theta02^3*theta03^2 + x01*x05*theta00*theta01*theta02^3*theta03^2 - x01*x04*theta00^2*theta01*theta02*theta03^3 + x02*x04*theta00^2*theta01*theta02*theta03^3 + x01*x05*theta00^2*theta01*theta02*theta03^3 - x02*x05*theta00^2*theta01*theta02*theta03^3 + x00*x04*theta00*theta01^2*theta02*theta03^3 - x02*x04*theta00*theta01^2*theta02*theta03^3 - x00*x05*theta00*theta01^2*theta02*theta03^3 + x02*x05*theta00*theta01^2*theta02*theta03^3 - x00*x04*theta00*theta01*theta02^2*theta03^3 + x01*x04*theta00*theta01*theta02^2*theta03^3 + x00*x05*theta00*theta01*theta02^2*theta03^3 - x01*x05*theta00*theta01*theta02^2*theta03^3 - x02*x03*theta00^3*theta01^2*theta02*theta04 + x03*x04*theta00^3*theta01^2*theta02*theta04 + x02*x05*theta00^3*theta01^2*theta02*theta04 - x04*x05*theta00^3*theta01^2*theta02*theta04 + x02*x03*theta00^2*theta01^3*theta02*theta04 - x03*x04*theta00^2*theta01^3*theta02*theta04 - x02*x05*theta00^2*theta01^3*theta02*theta04 + x04*x05*theta00^2*theta01^3*theta02*theta04 + x01*x03*theta00^3*theta01*theta02^2*theta04 - x03*x04*theta00^3*theta01*theta02^2*theta04 - x01*x05*theta00^3*theta01*theta02^2*theta04 + x04*x05*theta00^3*theta01*theta02^2*theta04 - x00*x03*theta00*theta01^3*theta02^2*theta04 + x03*x04*theta00*theta01^3*theta02^2*theta04 + x00*x05*theta00*theta01^3*theta02^2*theta04 - x04*x05*theta00*theta01^3*theta02^2*theta04 - x01*x03*theta00^2*theta01*theta02^3*theta04 + x03*x04*theta00^2*theta01*theta02^3*theta04 + x01*x05*theta00^2*theta01*theta02^3*theta04 - x04*x05*theta00^2*theta01*theta02^3*theta04 + x00*x03*theta00*theta01^2*theta02^3*theta04 - x03*x04*theta00*theta01^2*theta02^3*theta04 - x00*x05*theta00*theta01^2*theta02^3*theta04 + x04*x05*theta00*theta01^2*theta02^3*theta04 + x02*x03*theta00^3*theta01^2*theta03*theta04 - x02*x04*theta00^3*theta01^2*theta03*theta04 - x03*x05*theta00^3*theta01^2*theta03*theta04 + x04*x05*theta00^3*theta01^2*theta03*theta04 - x02*x03*theta00^2*theta01^3*theta03*theta04 + x02*x04*theta00^2*theta01^3*theta03*theta04 + x03*x05*theta00^2*theta01^3*theta03*theta04 - x04*x05*theta00^2*theta01^3*theta03*theta04 - x01*x03*theta00^3*theta02^2*theta03*theta04 + x01*x04*theta00^3*theta02^2*theta03*theta04 + x03*x05*theta00^3*theta02^2*theta03*theta04 - x04*x05*theta00^3*theta02^2*theta03*theta04 + x00*x03*theta01^3*theta02^2*theta03*theta04 - x00*x04*theta01^3*theta02^2*theta03*theta04

{((0,),
  (2,
   2,
   1,
   1)): (-theta00^3*theta01*theta02^2*theta04 + theta00^2*theta01*theta02^3*theta04 + theta00^3*theta02^2*theta03*theta04 - theta00^2*theta02^3*theta03*theta04 + theta00^3*theta01*theta02*theta04^2 - theta00*theta01*theta02^3*theta04^2 - theta00^3*theta02*theta03*theta04^2 + theta00*theta02^3*theta03*theta04^2 - theta00^2*theta01*theta02*theta04^3 + theta00*theta01*theta02^2*theta04^3 + theta00^2*theta02*theta03*theta04^3 - theta00*theta02^2*theta03*theta04^3 + theta00^3*theta01*theta02^2*theta05 - theta00^2*theta01*theta02^3*theta05 - theta00^3*theta02^2*theta03*theta05 + theta00^2*theta02^3*theta03*theta05 - theta00^3*theta01*theta04^2*theta05 + theta01*theta02^3*theta04^2*theta05 + theta00^3*theta03*theta04^2*theta05 - theta02^3*theta03*theta04^2*theta05 + theta00^2*theta01*theta04^3*theta05 - theta01*theta02^2*theta04^3*theta05 - theta00^2*theta03*theta04^3*theta05 + theta02^2*theta03*theta04^3*theta05 - theta00^3*theta01*theta02*theta05^2 + theta00*theta01*theta02^3*theta05^2 + theta00^3*theta02*theta03*theta05^2 - theta00*theta02^3*theta03*theta05^2 + theta00^3*theta01*theta04*theta05^2 - theta01*theta02^3*theta04*theta05^2 - theta00^3*theta03*theta04*theta05^2 + theta02^3*theta03*theta04*theta05^2 - theta00*theta01*theta04^3*theta05^2 + theta01*theta02*theta04^3*theta05^2 + theta00*theta03*theta04^3*theta05^2 - theta02*theta03*theta04^3*theta05^2 + theta00^2*theta01*theta02*theta05^3 - theta00*theta01*theta02^2*theta05^3 - theta00^2*theta02*theta03*theta05^3 + theta00*theta02^2*theta03*theta05^3 - theta00^2*theta01*theta04*theta05^3 + theta01*theta02^2*theta04*theta05^3 + theta00^2*theta03*theta04*theta05^3 - theta02^2*theta03*theta04*theta05^3 + theta00*theta01*theta04^2*theta05^3 - theta01*theta02*theta04^2*theta05^3 - theta00*theta03*theta04^2*theta05^3 + theta02*theta03*theta04^2*theta05^3,)}

s[] # s[2, 2, 1, 1] + s[1] # s[2, 1, 1, 1, 1] + s[2] # s[1, 1, 1, 1, 1, 1]

### Latex factorize output for characters

In [87]:
def latex_output_character(f):
    n = c.support().pop()[1].size()
    tensor = sorted(factorise(f, n), reverse=True)
    output = ''
    for a,b in tensor:
        if b != 0 :
            if b == s([]) or b == 1:
                b = 1
                output += str(b)
            elif len(b) > 1:
                output += "(%s)"%latex(b)
            else:
                output += latex(b)
            output += " \otimes %s +"%latex(s(a))
    output = output[:len(output)-1]
    return output

In [98]:
for mu in Partitions(3):
    print(latex_output_character(compute_character(mu)))

1 \otimes s_{3} +(s_{1} + s_{2}) \otimes s_{2,1} +(s_{1,1} + s_{3}) \otimes s_{1,1,1} 
1 \otimes s_{2,1} + s_{1} \otimes s_{1,1,1} 
1 \otimes s_{1,1,1} 


### Implémentation formule de François

Ne fonctionne que pour un jeu de variables.

In [271]:
Sym = SymmetricFunctions(FractionField(QQ['q','t']))
Sym.inject_shorthands(verbose=False)
Ht = Sym.macdonald().Ht(); Ht

Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in q, t over Rational Field in the Macdonald Ht basis

In [272]:
s(Ht([1,1,1]))

t^3*s[1, 1, 1] + (t^2+t)*s[2, 1] + s[3]

In [273]:
s(Ht([2,1]))

q*t*s[1, 1, 1] + (q+t)*s[2, 1] + s[3]

In [274]:
s(Ht([3]))

q^3*s[1, 1, 1] + (q^2+q)*s[2, 1] + s[3]